In [4]:
import numpy as np
import tensorflow as tf
from IPython import display

#### Code for dissecting each word

In [5]:
endings = ['ություն', 'ության', 'ությամբ', 'ներ', 'ով', 'եր', 'իկ', '-' 'ը', 'ի', 'ն']
endings = sorted(endings, key=len)[::-1]

def dissect_word(word):
    ans = []
    repeat_while=True
    flag_one=False
    
    for _ in range(5):
        repeat_loop = False
        
        for e in endings:
            if word[-len(e):] == e:
                repeat_loop=True
                        
                ans.append(e)
                word = word[:-len(e)]
                break
                
        if not repeat_loop:
            break
    
    if word!='':
        if 'չ' == word[0]:
            ans.append(word[1:])
            ans.append('չ')
        elif 'ան' == word[:2]:
            ans.append(word[2:])
            ans.append('ան')
        else:
            ans.append(word)
            
    return  ans[::-1]

# First lets connect the word pairs and seperate numbers from each other

In [13]:
file_names = ['corpus_100k', 'corpus_100k_pass1', 'corpus_100k_pass2']

max_iter=2
for iter_ in range(max_iter):
    vocab = dict()
    pair_vocab = dict()
    line_num = 0
    
    # loading data
    with open(file_names[iter_], 'r', encoding="utf8") as file:
        line = file.readline()

        while line:
            line_num+=1
            if line_num%10000==0:
                print(f'iter = {iter_}\nloading... line_num = {line_num}')
                display.clear_output(wait=True)

            if iter_ ==0:
                line = line.lower()
            line = line.split()
                
            prevs=''
            for token in line:
                if prevs!='':
                    pair_w = (prevs, token)
                    if pair_w in pair_vocab:
                        pair_vocab[pair_w]+=1
                    else:
                        pair_vocab[pair_w]=1
                prevs=token

                if token in vocab:
                    vocab[token] += 1
                else:
                    vocab[token] = 1

            line = file.readline()
        
    # calculating pair-scores
    for k in pair_vocab.keys():
        pair_vocab[k]-=20
        pair_vocab[k]/=vocab[k[0]]*vocab[k[1]]
    
    # sorting in descending order
    pair_vocab = sorted(pair_vocab.items(), key=lambda x: -x[1])
    pair_vocab = [x[0] for x in pair_vocab]

    # saving processed data
    line_num = 0
    f = open(file_names[iter_+1], 'w', encoding="utf8")
    with open(file_names[iter_], 'r', encoding="utf8") as file:
        line = file.readline()

        while line:
            line_num+=1
            if line_num%10000==0:
                print(f'iter = {iter_}\nsaving... line_num = {line_num}')
                display.clear_output(wait=True)
            
            
            if iter_==0:
                line = line.lower().split()
            
            elif iter_ == max_iter-1:
                # separating all digits
                for i in range(10):
                    line = line.replace(str(i), f' {i} ')
                line = line.replace('   ', ' ').replace('  ', ' ')
                line = line.split()
            
            else:
                line = line.split()

            token_indx=1
            while token_indx < len(line):
                prevs = line[token_indx-1]
                token = line[token_indx]
                pair_w = (prevs, token)
                if pair_w in pair_vocab[:850]:
                    if iter_ == max_iter-1:
                        f.write(f'{prevs}_{" ".join(dissect_word(token))} ')
                    else:
                        f.write(f'{prevs}_{token} ')
                    token_indx+=1
                else:
                    if iter_ == max_iter-1:
                        f.write(f'{" ".join(dissect_word(prevs))} ')
                    else:
                        f.write(f'{prevs} ')

                token_indx+=1

            if token_indx==len(line)+1:
                f.write('\n')
            else:
                f.write(f'{line[-1]}\n')

            line = file.readline()
    f.close()

print('done')

done
